In [1]:
import numpy as np
import pandas as pd



In [2]:
# initialize fees
FEE_F = 0.003

# initialize basic arrays and dictionaries
q = {'ab': [0., 0.], 'ac': [0., 0.], 'bc': [0.,0.]}
swap_amt = [0.]*2

# Test swap method, or not
TEST_SWAP_CODE = True

In [3]:
# Initialize dataframe

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [200., 200.]
    q_['bc'] = [250., 250.]
    return q_

q = reset_reserves(q)

init_state = {
    'exch': ['0'],
    'tok0': [0.],
    'tok1': [0.],
    'res0in': [0.],
    'res1in': [0.],
    'res0out': [0.],
    'res1out': [0.]
}

ledger_df = pd.DataFrame(init_state)

print(ledger_df)


  exch  tok0  tok1  res0in  res1in  res0out  res1out
0    0   0.0   0.0     0.0     0.0      0.0      0.0


In [4]:
# Swap code
#     Follows the Uniswap code

def psi(x):
    assert x >= 0
    return 1/(1+x)

def phi(x):
    assert x >= 0
    assert x < 1
    return 1/(1-x)

def swap(data_):
    # data_ = { 'reserves': q['ab'], 'legs': [10., 0.], 'direction': 'forward', 'fee': FEE_F}
    f_ = data_['fee']
    d_ = data_['legs']
    r_ = data_['reserves']
    direction:str = data_['direction']

    assert r_[0] > 0
    assert r_[1] > 0

    # The in_leg has the non-zero entry
    if d_[0] > 0:
        assert d_[1] == 0
        in_leg = 0
        out_leg = 1
    else:
        assert d_[0] == 0
        in_leg = 1
        out_leg = 0

    q_in = r_[in_leg]
    q_out = r_[out_leg]

    if direction == 'backward':
        # backward swap:
        #     find the input that produces the output
        #     the output is known, so this is the "in_leg" which has a non-zero entry
        d_out = d_[in_leg]
        d_in = float((1 / (1 - f_)) * (q_in / q_out) * phi(d_out / q_out) * d_out)
        result = d_in
    elif direction == 'forward':
        # forward swap:
        #     find the output produced by the input
        #     the input is known, so this is the "in_leg" which has a non-zero entry
        d_in = d_[in_leg]
        d_in_haircut = (1 - f_) * d_in
        d_out = float((q_out / q_in) * psi(d_in_haircut / q_in) * d_in_haircut)
        d_[out_leg] = d_out
        result = d_out
    else:
        print('unknown direction')
        return

    r_[out_leg] -= d_out
    r_[in_leg]  += d_in
    return {'reserves_in': [q_in, q_out], 'reserves_out': r_, 'legs': [d_in, d_out], 'result': result }


# Tests
if TEST_SWAP_CODE:
    q = reset_reserves(q)

    # swap a->b
    reason = 'route1: a->b'
    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [10., 0.],
                  'direction': 'forward',
                  'fee': FEE_F})
    print(reason)
    print(swap_out)
    route1_b_tokens = swap_out['result']

    # swap c->b
    reason = 'cycle: c->b '
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [route1_b_tokens, 0.],
                  'direction': 'backward',
                  'fee': FEE_F})
    cycle_c1_tokens = swap_out['result']
    print(reason)
    print(swap_out)
    print(f"cycle_c1_tokens: {cycle_c1_tokens}")


route1: a->b
{'reserves_in': [100.0, 100.0], 'reserves_out': [110.0, 90.93389106119851], 'legs': [10.0, 9.066108938801491], 'result': 9.066108938801491}
cycle: c->b 
{'reserves_in': [250.0, 250.0], 'reserves_out': [259.4355645298258, 240.9338910611985], 'legs': [9.435564529825822, 9.066108938801491], 'result': 9.435564529825822}
cycle_c1_tokens: 9.435564529825822


In [5]:
def run_compare_test(q):
    # Run two routes: a->b and a->c->b
    # Notice that a->c->b returns *more* b-tokens

    # a->b route1
    q = reset_reserves(q)

    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [10., 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    route1_b_tokens = swap_out['result']

    print(f"route1_b_tokens: {route1_b_tokens}")

    # a->c->b route2
    q = reset_reserves(q)

    # swap a->c
    exch = 'ac'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [10., 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    route2_c_tokens = swap_out['result']
    print(f"route2_c_tokens: {route2_c_tokens}")

    # swap c->b
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [0., route2_c_tokens], # note: incoming leg is c-tokens, which is index 1
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    route2_b_tokens = swap_out['result']
    print(f"route2_b_tokens: {route2_b_tokens}")

    print(f"Difference b-tokens route1 minus route2: {route1_b_tokens - route2_b_tokens}")

    # Return the b-tokens from the more expensive route
    return route1_b_tokens

In [19]:

def run_cycle_test(q, cycle_b_tokens):
    # Run cycle: c1->b->a->c2
    # Notice that c2 > c1

    q = reset_reserves(q)


    # swap c->b
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [cycle_b_tokens, 0.],
                  'direction': 'backward',
                  'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    cycle_c1_tokens = swap_out['result']
    print(f"cycle_c1_tokens: {cycle_c1_tokens}")
    print(f"cycle_b_tokens: {route1_b_tokens}")

    # swap b->a
    reason = 'cycle: b->a'
    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [0., route1_b_tokens],
                      'direction': 'backward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    cycle_a_tokens = swap_out['result']
    print(f"cycle_a_tokens: {cycle_a_tokens}")

    # swap a->c
    reason = 'cycle: a->c'
    exch = 'ac'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [cycle_a_tokens, 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    cycle_c2_tokens = swap_out['result']
    print(f"cycle_c2_tokens: {cycle_c2_tokens}")

    print(f"Difference c2 minus c1: {cycle_c2_tokens - cycle_c1_tokens}")


In [7]:
def run_post_trade_arb_test(q):

    # a->b route1
    q = reset_reserves(q)

    reason = 'route1: b->a'
    exch = 'ab'

    route1_b_tokens = 10.
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [0, route1_b_tokens],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    route1_a_tokens = swap_out['result']

    print(f"route1_a_tokens: {route1_a_tokens}")
    print(f"route1_b_tokens: {route1_b_tokens}")

    # run cycle to immunize a->b loss by c-token profit

    # swap c->b
    reason = 'cycle: c->b'
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [route1_b_tokens, 0.],
                  'direction': 'backward',
                  'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    cycle_c1_tokens = swap_out['result']

    print(f"cycle_c1_tokens: {cycle_c1_tokens}")
    print(f"cycle_b_tokens: {route1_b_tokens}")

    # swap b->a
    reason = 'cycle: b->a'
    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [0., route1_b_tokens],
                      'direction': 'backward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    cycle_a_tokens = swap_out['result']

    print(f"cycle_a_tokens: {cycle_a_tokens}")

    # swap a->c
    reason = 'cycle: a->c'
    exch = 'ac'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [cycle_a_tokens, 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    cycle_c2_tokens = swap_out['result']
    print(f"cycle_c2_tokens: {cycle_c2_tokens}")

    print(f"Difference c2 minus c1: {cycle_c2_tokens - cycle_c1_tokens}")


In [13]:
# Experiment 1
#   Compare routes
#   The route using more liquidity has better prices

experiment = 'exp1'
def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [200., 200.]
    q_['bc'] = [250., 250.]
    return q_

q = reset_reserves(q)

nrow = len(ledger_df)
route1_b_tokens = run_compare_test(q)
print(ledger_df[nrow:])

route1_b_tokens: 9.066108938801491
route2_c_tokens: 9.496594751631186
route2_b_tokens: 9.122609663880215
Difference b-tokens route1 minus route2: -0.056500725078723946
   exch       tok0      tok1  res0in  res1in    res0out     res1out
14   ab  10.000000  9.066109   100.0   100.0  110.00000   90.933891
15   ac  10.000000  9.496595   200.0   200.0  210.00000  190.503405
16   bc   9.496595  9.122610   250.0   250.0  240.87739  259.496595


In [14]:
# Experiment 2
#   Test if a cycle leveraging cheaper route can generate arbitrage cycle
#   Result: yes, but effect is minor
#   Difference c2 minus c1: 0.06103022180536222

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [200., 200.]
    q_['bc'] = [250., 250.]
    return q_

q = reset_reserves(q)

nrow = len(ledger_df)
run_cycle_test(q, route1_b_tokens)
print(ledger_df[nrow:])

cycle_c1_tokens: 9.435564529825822
cycle_b_tokens: 9.066108938801491
cycle_a_tokens: 9.999999999999998
cycle_c2_tokens: 9.496594751631184
Difference c2 minus c1: 0.06103022180536222
SUCCESS: cycle arbitrage from mispricing b-tokens
   exch       tok0      tok1  res0in  res1in     res0out     res1out
17   bc   9.435565  9.066109   250.0   250.0  259.435565  240.933891
18   ab  10.000000  9.066109   100.0   100.0   90.933891  110.000000
19   ac  10.000000  9.496595   200.0   200.0  210.000000  190.503405


In [15]:
# Experiment 3
#   Test cycle arbitrage after a trade on more expensive exchange
#   It is!
#   Difference c2 minus c1: 2.2752286620613518

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [200., 200.]
    q_['bc'] = [250., 250.]
    return q_

q = reset_reserves(q)
nrow = len(ledger_df)
run_post_trade_arb_test(q)
print(ledger_df[nrow:])

route1_a_tokens: 9.066108938801491
route1_b_tokens: 10.0
cycle_c1_tokens: 10.448010698762957
cycle_b_tokens: 10.0
cycle_a_tokens: 13.632236326745932
cycle_c2_tokens: 12.726489418614255
Difference c2 minus c1: 2.2784787198512984
   exch       tok0       tok1  res0in      res1in     res0out     res1out
20   ab  10.000000   9.066109   100.0  100.000000   90.933891  110.000000
21   bc  10.448011  10.000000   250.0  250.000000  260.448011  240.000000
22   ab  13.632236  10.000000   110.0   90.933891   80.933891  123.632236
23   ac  13.632236  12.726489   200.0  200.000000  213.632236  187.273511


In [21]:
# Experiment 3.5
#   Test cycle arbitrage after a trade on more expensive exchange
#   Scale c-tokens 10x
#   A bit more, only 10% ish
#   Difference c2 minus c1: 3.4292290304131523

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [2000., 2000.]
    q_['bc'] = [2500., 2500.]
    return q_

q = reset_reserves(q)
nrow = len(ledger_df)
run_post_trade_arb_test(q)
print(ledger_df[nrow:])

route1_a_tokens: 9.066108938801491
route1_b_tokens: 10.0
cycle_c1_tokens: 10.070371757843812
cycle_b_tokens: 10.0
cycle_a_tokens: 13.632236326745932
cycle_c2_tokens: 13.499600788256965
Difference c2 minus c1: 3.4292290304131523
   exch       tok0       tok1  res0in       res1in      res0out      res1out
36   ab  10.000000   9.066109   100.0   100.000000    90.933891   110.000000
37   bc  10.070372  10.000000  2500.0  2500.000000  2510.070372  2490.000000
38   ab  13.632236  10.000000   110.0    90.933891    80.933891   123.632236
39   ac  13.632236  13.499601  2000.0  2000.000000  2013.632236  1986.500399


In [17]:
# Experiment 4
#   Same as Experiment 1
#   Test if 10x scale makes any difference
#   It does but not much more:
#   Difference b-tokens route1 minus route2: -0.7856986586526489

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [2000., 2000.]
    q_['bc'] = [2500., 2500.]
    return q_

q = reset_reserves(q)

nrow = len(ledger_df)
route1_b_tokens = run_compare_test(q)
print(ledger_df[nrow:])

route1_b_tokens: 9.066108938801491
route2_c_tokens: 9.920546077802156
route2_b_tokens: 9.85180759745414
Difference b-tokens route1 minus route2: -0.7856986586526489
   exch       tok0      tok1  res0in  res1in      res0out      res1out
27   ab  10.000000  9.066109   100.0   100.0   110.000000    90.933891
28   ac  10.000000  9.920546  2000.0  2000.0  2010.000000  1990.079454
29   bc   9.920546  9.851808  2500.0  2500.0  2490.148192  2509.920546


In [20]:
# Experiment 5
#   Test if a cycle leveraging cheaper route can generate arbitrage cycle
#   Scale by 10x on c-tokens
#   Result: yes, effect is minor
#   Difference c2 minus c1: 0.7940602859152346

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [2000., 2000.]
    q_['bc'] = [2500., 2500.]
    return q_

q = reset_reserves(q)

nrow = len(ledger_df)
run_cycle_test(q, route1_b_tokens)
print(ledger_df[nrow:])

cycle_c1_tokens: 9.12648579188692
cycle_b_tokens: 9.066108938801491
cycle_a_tokens: 9.999999999999998
cycle_c2_tokens: 9.920546077802154
Difference c2 minus c1: 0.7940602859152346
   exch       tok0      tok1  res0in  res1in      res0out      res1out
33   bc   9.126486  9.066109  2500.0  2500.0  2509.126486  2490.933891
34   ab  10.000000  9.066109   100.0   100.0    90.933891   110.000000
35   ac  10.000000  9.920546  2000.0  2000.0  2010.000000  1990.079454
